In [1]:
import pandas as pd
import numpy as np

In [118]:
from cluster_functions import load_vdjdb
import os
vdjdb = load_vdjdb(os.path.join(os.getcwd(),'data/vdjdb_full.txt'))

In [35]:
nodelist = '/Users/danhudson/Documents/Academic/Oxford/Oxford_DPhil/Software/clusTCR/tcr_compare/cytoscape/20220626_120653_typhi.csv/clustcr_beta_typhi.csv_nodelist.txt'

In [36]:
nodes = pd.read_csv(nodelist)